## Full Extraction
Full extraction refers to extracting **all** data from a source system each time the extraction process runs, regardless of whether the data has changed or not.

**Characteristics:**
- Simpler to implement as it doesn't require tracking changes
- Guarantees data consistency between source and target
- Resource-intensive (requires processing all data every time)
- Slower, especially with large datasets
- May require more storage space

**Use cases:** Small datasets, when source systems don't support change tracking, initial loads

Let's fully extract all the records found in our source.

In [3]:
import pandas as pd

# Load full dataset
full_df = pd.read_csv("google_5yr_one.csv")

# Display basic stats
print(f"Extracted {len(full_df)} rows fully.")
print("Shape:", full_df.shape)

Extracted 1256 rows fully.
Shape: (1256, 6)


## Incremental Extraction
Incremental extraction only retrieves data that has **changed** since the last extraction.

**Characteristics:**
- More complex to implement (requires change tracking)
- More efficient (processes only changed data)
- Faster execution
- Less resource-intensive
- Requires reliable change tracking mechanisms

**Types of incremental extraction:**
1. **Date/time-based:** Uses timestamp columns
2. **Version number-based:** Uses version or sequence numbers
3. **Log-based:** Reads database transaction logs
4. **Trigger-based:** Uses database triggers to track changes

**Use cases:** Large datasets, frequent updates, when source systems support change tracking

First, let's create a text file to track our last date of extraction. Let say the last time we extracted data was on January 1st, 2025.

In [5]:
from datetime import datetime

# Save last extraction date timestamp to last_extraction.txt
last_extraction = datetime(2025, 1, 1, 0, 0, 0).strftime("%Y-%m-%d %H:%M:%S")
with open("last_extraction.txt", "w") as f:
    f.write(last_extraction)
print("Last extraction time saved as 'last_extraction.txt'")

Last extraction time saved as 'last_extraction.txt'


**Now, we can extract the data recorded after our last time of extraction, which is January 1st, 2025.**

In [7]:
# Read last extraction timestamp
with open("last_extraction.txt", "r") as f:
    last_extraction_str = f.read().strip()

last_extraction = datetime.strptime(last_extraction_str, "%Y-%m-%d %H:%M:%S")

data = pd.read_csv("google_5yr_one.csv")

# Convert 'Date' column to datetime
data["Date"] = pd.to_datetime(data["Date"])

# Filter for rows after last extraction
incremental_df = data[data["Date"] > last_extraction]

print(f"Extracted {len(incremental_df)} rows incrementally since '{last_extraction}'.")
print("Shape:", incremental_df.shape)

Extracted 103 rows incrementally since '2025-01-01 00:00:00'.
Shape: (103, 6)


**Update the last time of extraction.**

In [9]:
# Save current timestamp to last_extraction.txt
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
with open("last_extraction.txt", "w") as f:
    f.write(now)

print(f"Last extraction date updated to '{now}'.")

Last extraction date updated to '2025-06-16 00:25:15'.


### Full Data Transformation 

In [11]:
full_df.head()

,Date,Close,High,Low,Open,Volume
0,NaN,GOOGL,GOOGL,GOOGL,GOOGL,GOOGL
1,2020-06-04,70.3785171508789,71.72309429138843,69.96599205492319,71.4971694316438,26982000
2,2020-06-05,71.65840148925781,71.9709103787135,70.0461071028752,70.44520002096422,42642000
3,2020-06-08,72.05748748779297,72.10525562528537,70.88509140875318,70.974667107052,33878000
4,2020-06-09,72.25852966308594,73.04079279119881,71.77484210279437,71.91816171630913,33624000


The first record (index 0) appears to be invalid/malformed data with these characteristics:

Date: NaN (pandas' representation of missing values)

All other columns: Contain the string "GOOGL" instead of numerical values

Volume: Also shows "GOOGL" instead of a numeric volume count

This is likely a header row that was incorrectly parsed as data during CSV import

In [13]:
# Remove the first record
full_df = full_df.iloc[1:]
full_df.head()

,Date,Close,High,Low,Open,Volume
1,2020-06-04,70.3785171508789,71.72309429138843,69.96599205492319,71.4971694316438,26982000
2,2020-06-05,71.65840148925781,71.9709103787135,70.0461071028752,70.44520002096422,42642000
3,2020-06-08,72.05748748779297,72.10525562528537,70.88509140875318,70.974667107052,33878000
4,2020-06-09,72.25852966308594,73.04079279119881,71.77484210279437,71.91816171630913,33624000
5,2020-06-10,72.88652801513672,73.28810385800969,72.37198974644225,72.72778683921254,31762000


In [14]:
# Check data types of the columns
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1255 entries, 1 to 1255
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    1255 non-null   object
 1   Close   1255 non-null   object
 2   High    1255 non-null   object
 3   Low     1255 non-null   object
 4   Open    1255 non-null   object
 5   Volume  1255 non-null   object
dtypes: object(6)
memory usage: 59.0+ KB


Each column has incorrect data type.

Let's change their types into their appropriate ones.

In [16]:
full_df["Date"] = pd.to_datetime(full_df["Date"])
full_df["Close"] = full_df["Close"].astype("float")
full_df["High"] = full_df["High"].astype("float")
full_df["Low"] = full_df["Low"].astype("float")
full_df["Open"] = full_df["Open"].astype("float")
full_df["Volume"] = full_df["Volume"].astype("int")

full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1255 entries, 1 to 1255
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    1255 non-null   datetime64[ns]
 1   Close   1255 non-null   float64       
 2   High    1255 non-null   float64       
 3   Low     1255 non-null   float64       
 4   Open    1255 non-null   float64       
 5   Volume  1255 non-null   int32         
dtypes: datetime64[ns](1), float64(4), int32(1)
memory usage: 54.1 KB


In [17]:
# Check for duplicates
full_df.duplicated().sum()

0

As we can see from the output, we don't have any duplicated values.

In [19]:
# Check for missing values
full_df.isnull().sum().sum()

0

Our data doesn't contain missing values.

In [21]:
# Add a new column: price_range based on 'Close' price
bins = [0, 100, 150, 200, float('inf')]
labels = ['Low', 'Medium', 'High', 'Very High']
full_df['price_range'] = pd.cut(full_df['Close'], bins=bins, labels=labels)
full_df.head(10)

,Date,Close,High,Low,Open,Volume,price_range
1,2020-06-04,70.378517,71.723094,69.965992,71.497169,26982000,Low
2,2020-06-05,71.658401,71.970910,70.046107,70.445200,42642000,Low
3,2020-06-08,72.057487,72.105256,70.885091,70.974667,33878000,Low
4,2020-06-09,72.258530,73.040793,71.774842,71.918162,33624000,Low
5,2020-06-10,72.886528,73.288104,72.371990,72.727787,31762000,Low
6,2020-06-11,69.761475,72.234651,69.587805,71.708662,47144000,Low
7,2020-06-12,70.309853,71.384214,68.960302,70.953770,36676000,Low
8,2020-06-15,70.698990,70.910974,69.029968,69.143925,30468000,Low
9,2020-06-16,71.979370,72.511821,71.109528,72.105264,30648000,Low
10,2020-06-17,72.281425,72.762129,71.415069,72.301335,30610000,Low


In [22]:
# Save transformed data
full_df.to_csv("Download/transformed_full.csv", index=False)

# Preview transformed full data
full_df.head()

,Date,Close,High,Low,Open,Volume,price_range
1,2020-06-04,70.378517,71.723094,69.965992,71.497169,26982000,Low
2,2020-06-05,71.658401,71.970910,70.046107,70.445200,42642000,Low
3,2020-06-08,72.057487,72.105256,70.885091,70.974667,33878000,Low
4,2020-06-09,72.258530,73.040793,71.774842,71.918162,33624000,Low
5,2020-06-10,72.886528,73.288104,72.371990,72.727787,31762000,Low


### Incremental Data Transformation

In [24]:
incremental_df.head()

,Date,Close,High,Low,Open,Volume
1153,2025-01-02,189.2120819091797,191.7791328258186,187.2843094002135,190.43067970164262,20370800
1154,2025-01-03,191.56936645507812,192.9877463613902,189.76145103861666,191.14985143368122,18596200
1155,2025-01-06,196.64352416992188,197.99197728329852,193.6270092243641,193.75684931380115,29563600
1156,2025-01-07,195.26512145996094,200.7687774852745,194.37614589069548,196.8832529983184,26487200
1157,2025-01-08,193.7268829345703,196.06418741382137,192.1586969389352,192.3484808074723,24864800


In [25]:
# Check data types of the columns
incremental_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103 entries, 1153 to 1255
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    103 non-null    datetime64[ns]
 1   Close   103 non-null    object        
 2   High    103 non-null    object        
 3   Low     103 non-null    object        
 4   Open    103 non-null    object        
 5   Volume  103 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 5.6+ KB


Date has the correct data type which is datetime but the rest have wrong data types.

Let's change their types into their appropriate ones.

In [53]:
incremental_df.loc[:,"Close"] = incremental_df["Close"].astype("float")
incremental_df.loc[:,"High"] = incremental_df["High"].astype("float")
incremental_df.loc[:,"Low"] = incremental_df["Low"].astype("float")
incremental_df.loc[:,"Open"] = incremental_df["Open"].astype("float")
incremental_df.loc[:,"Volume"] = incremental_df["Volume"].astype("int")

incremental_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103 entries, 1153 to 1255
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         103 non-null    datetime64[ns]
 1   Close        103 non-null    float64       
 2   High         103 non-null    float64       
 3   Low          103 non-null    float64       
 4   Open         103 non-null    float64       
 5   Volume       103 non-null    int32         
 6   price_range  103 non-null    category      
dtypes: category(1), datetime64[ns](1), float64(4), int32(1)
memory usage: 5.5 KB


In [28]:
# Check for duplicates
incremental_df.duplicated().sum()

0

Our data does not contain duplicated values.

In [30]:
# Check for missing values
incremental_df.isnull().sum().sum()

0

We do not have any missing values.

In [55]:
# Add a new column: price_range based on 'Close' price
bins = [0, 100, 150, 200, float('inf')]
labels = ['Low', 'Medium', 'High', 'Very High']
incremental_df.loc[:,'price_range'] = pd.cut(incremental_df['Close'], bins=bins, labels=labels)
incremental_df.head(10)

,Date,Close,High,Low,Open,Volume,price_range
1153,2025-01-02,189.212082,191.779133,187.284309,190.430680,20370800,High
1154,2025-01-03,191.569366,192.987746,189.761451,191.149851,18596200,High
1155,2025-01-06,196.643524,197.991977,193.627009,193.756849,29563600,High
1156,2025-01-07,195.265121,200.768777,194.376146,196.883253,26487200,High
1157,2025-01-08,193.726883,196.064187,192.158697,192.348481,24864800,High
1158,2025-01-10,191.819077,196.293934,190.091071,194.076486,26665200,High
1159,2025-01-13,190.790268,190.960071,187.144473,189.851362,21823700,High
1160,2025-01-14,189.441818,191.759141,188.093365,191.020002,17174900,High
1161,2025-01-15,195.325043,196.134108,191.639285,192.867866,21776000,High
1162,2025-01-16,192.688080,195.255115,192.588189,193.916661,17815400,High


In [33]:
# Save transformed data
incremental_df.to_csv("Download/transformed_incremental.csv", index=False)

# Preview transformed full data
incremental_df.head()

,Date,Close,High,Low,Open,Volume,price_range
1153,2025-01-02,189.212082,191.779133,187.284309,190.430680,20370800,High
1154,2025-01-03,191.569366,192.987746,189.761451,191.149851,18596200,High
1155,2025-01-06,196.643524,197.991977,193.627009,193.756849,29563600,High
1156,2025-01-07,195.265121,200.768777,194.376146,196.883253,26487200,High
1157,2025-01-08,193.726883,196.064187,192.158697,192.348481,24864800,High
